# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import json
import pandas as pd
import numpy as np
from datetime import datetime as dt, timedelta
import re
from sql_queries import *

In [2]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the Database")
    print(e)


In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
for filepath in song_files:
    df = pd.read_json(filepath, lines=True)
    df = pd.DataFrame(df)

#filepath = song_files[0]


In [6]:
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AREVWGE1187B9B890A,-13.442,Noci (BA),-41.9952,Bitter End,282.43546,1,SOFCHDR12AB01866EF,Living Hell,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
songlist = df[['song_id','title','artist_id','year','duration']]
song_data = songlist.values[0].tolist()
song_data

['SOFCHDR12AB01866EF', 'Living Hell', 'AREVWGE1187B9B890A', 0, 282.43546]

In [8]:
songlist

,song_id,title,artist_id,year,duration
0,SOFCHDR12AB01866EF,Living Hell,AREVWGE1187B9B890A,0,282.43546


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
for filepath in song_files:
        with open(filepath) as f:
            data=json.load(f)
            df = pd.DataFrame(data, index=[0])
            df.head()

In [11]:
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AREVWGE1187B9B890A,-13.442,-41.9952,Noci (BA),Bitter End,SOFCHDR12AB01866EF,Living Hell,282.43546,0


In [12]:
df =df.rename(columns=
              {
                  "artist_id" : "artist_id",
                  "artist_name" : "name",
                  "artist_location" : "location",
                  "artist_latitude" : "latitude",
                  "artist_longitude" : "longitude"
              })

In [13]:
df

,num_songs,artist_id,latitude,longitude,location,name,song_id,title,duration,year
0,1,AREVWGE1187B9B890A,-13.442,-41.9952,Noci (BA),Bitter End,SOFCHDR12AB01866EF,Living Hell,282.43546,0


In [14]:
artistlist = df[['artist_id','name', 'location', 'latitude', 'longitude']]
artist_data = artistlist.values[0]
artist_data

array(['AREVWGE1187B9B890A', 'Bitter End', 'Noci (BA)', -13.442, -41.9952], dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files = get_files("data/log_data/2018")


In [17]:
for filepath in log_files:
    df = pd.read_json(filepath, lines=True)
    df


In [18]:
df.head(3)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
nextsong = [df.loc[df['page'] =='NextSong']]

In [20]:
nxtsong = nextsong[0]

In [21]:
t = nxtsong.ts

In [22]:
length=len(t)

In [23]:
timestamps = []
hour=[]
day=[]
month=[]
year=[]
#datetime64=dt
for time in t:
    dtime = dt.fromtimestamp(time/ 1000.0)
    d_series=('{0}-{1}-{2}'.format(dtime.year,dtime.month,dtime.day))
    #d =dtime.strftime("%Y %m %d %H:%M:%S.%f")
    d =dtime.strftime("%Y %m %d %H:%M:%S.%f")
    hour.append(dtime.hour)
    day.append(dtime.day)
    month.append(dtime.month)
    year.append(dtime.year)
    timestamps.append(d)
    tss= pd.Series(timestamps)
    
    

In [24]:
dayofweek_series =  pd.Series(pd.date_range(d_series, d_series, freq='D'))
wd=int(dayofweek_series.dt.weekday)
dow=int(dayofweek_series.dt.week)

In [25]:
dw=[]
w=[]
i=0
while i<length:
    dw.append(wd)
    i+=1
    
i=0
while i<length:
    w.append(dow)
    i+=1
    
    
dweek= np.array(dw).tolist()
week= np.array(w).tolist()
    

In [26]:
Array ={
    'start_time': tss,
    'hour':hour,
    'day': day,
    'week': week,
    'month': month,
    'year': year,
    'weekday': dweek}

df = pd.DataFrame(Array)

In [27]:
df.head()

,start_time,hour,day,week,month,year,weekday
0,2018 11 30 00:22:07.796000,0,30,48,11,2018,4
1,2018 11 30 01:08:41.796000,1,30,48,11,2018,4
2,2018 11 30 01:12:48.796000,1,30,48,11,2018,4
3,2018 11 30 01:17:05.796000,1,30,48,11,2018,4
4,2018 11 30 01:20:56.796000,1,30,48,11,2018,4


In [28]:
time_data = (df)
column_labels = df[['start_time', 'hour','day','week', 'month', 'year', 'weekday']]

In [29]:
time_df = time_data
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018 11 30 00:22:07.796000,0,30,48,11,2018,4
1,2018 11 30 01:08:41.796000,1,30,48,11,2018,4
2,2018 11 30 01:12:48.796000,1,30,48,11,2018,4
3,2018 11 30 01:17:05.796000,1,30,48,11,2018,4
4,2018 11 30 01:20:56.796000,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [30]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [31]:
log_files = get_files("data/log_data/2018")

In [32]:
for filepath in log_files:
    df = pd.read_json(filepath, lines=True)

In [33]:
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [34]:
df =df.rename(columns=
              {
                  "userId" : "user_id",
                  "firstName" : "first_name",
                  "lastName" : "last_name",
                  "gender" : "gender",
                  "level" : "level"
              })

In [35]:
#df.dropna(subset = ["user_id"], inplace=True)
df = df[df.user_id != ""]

In [36]:
df.head(3)

,artist,auth,first_name,gender,itemInSession,last_name,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,user_id
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [37]:
userlist = df[['user_id','first_name','last_name','gender','level']]
#user_data = userlist.values
#for userdata in user_data:
    #userdata

In [38]:
user_df = userlist.head(4)

In [39]:
user_df

,user_id,first_name,last_name,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [40]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [41]:
log_files = get_files("data/log_data/2018")

In [42]:
for filepath in log_files:
    df = pd.read_json(filepath, lines=True)
    
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [43]:
timestamp=[]
timestampdf = df.ts
timestampnp = np.array(timestampdf)
for tim in timestampnp:
    dtm = dt.fromtimestamp(tim/ 1000.0)
    dd =dtm.strftime("%Y %m %d %H:%M:%S.%f")
    timestamp.append(dd)
    tstamp= pd.Series(timestamp)
     
    
    

In [44]:
t = pd.to_datetime(df['ts'], unit='ms')

In [45]:
type(df['ts'])

pandas.core.series.Series

In [46]:
timestampdf = df.ts
timestampnp = np.array(timestampdf)
df['ts'].replace(timestampnp,tstamp, inplace =True)

In [47]:
df =df.rename(columns=
              {
                  "userId" : "user_id",
                  "sessionId" : "session_id",
                  "userAgent" : "user_agent",
                  "ts" : "start_time",
                  "level" : "level"
              })

df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,session_id,song,status,start_time,user_agent,user_id
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,2018 11 30 00:22:07.796000,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,2018 11 30 01:08:41.796000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,2018 11 30 01:12:48.796000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [48]:
#df["songplay_id"] = df.index + 1
#df.dropna(subset = ["user_id"], inplace=True)
df = df[df.user_id != ""]

In [49]:
#df = df[['artist','auth','firstName','gender','itemInSession','lastName','length','level','location' ,'method','page','registration','session_id','song','status','start_time','user_agent','user_id']]
#df = df[['start_time','user_id','level','session_id','location','user_agent']]
#dff = userlist.values
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,session_id,song,status,start_time,user_agent,user_id
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,2018 11 30 00:22:07.796000,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,2018 11 30 01:08:41.796000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,2018 11 30 01:12:48.796000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,2018 11 30 01:17:05.796000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,2018 11 30 01:20:56.796000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
5,LMFAO,Logged In,Jacob,M,4,Klein,227.99628,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,I'm In Miami Bitch,200,2018 11 30 01:24:32.796000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
6,DJ Dizzy,Logged In,Jacob,M,5,Klein,221.15220,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Sexy Bitch,200,2018 11 30 01:28:19.796000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
7,Fish Go Deep & Tracey K,Logged In,Jacob,M,6,Klein,377.41669,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,The Cure & The Cause (Dennis Ferrer Remix),200,2018 11 30 01:32:00.796000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
8,None,Logged In,Alivia,F,0,Terrell,NaN,free,"Parkersburg-Vienna, WV",GET,Home,1.540505e+12,1070,None,200,2018 11 30 01:34:04.796000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",4
9,M83,Logged In,Jacob,M,7,Klein,96.18240,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Staring At Me,200,2018 11 30 01:38:17.796000,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [70]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None
        
    
     # insert songplay record
    songplay_data = (row.start_time, row.user_id, row.level, song_id, artist_id,row.session_id, row.location, row.user_agent )
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

InterfaceError: cursor already closed

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [71]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [ ]:
%run etl.py

75 files found in data/song_data/A
1/75 files processed.
2/75 files processed.
3/75 files processed.
4/75 files processed.
5/75 files processed.
6/75 files processed.
7/75 files processed.
8/75 files processed.
9/75 files processed.
10/75 files processed.
11/75 files processed.
12/75 files processed.
13/75 files processed.
14/75 files processed.
15/75 files processed.
16/75 files processed.
17/75 files processed.
18/75 files processed.
19/75 files processed.
20/75 files processed.
21/75 files processed.
22/75 files processed.
23/75 files processed.
24/75 files processed.
25/75 files processed.
26/75 files processed.
27/75 files processed.
28/75 files processed.
29/75 files processed.
30/75 files processed.
31/75 files processed.
32/75 files processed.
33/75 files processed.
34/75 files processed.
35/75 files processed.
36/75 files processed.
37/75 files processed.
38/75 files processed.
39/75 files processed.
40/75 files processed.
41/75 files processed.
42/75 files processed.
43/75 fi